In [1]:
# default_exp products

In [2]:
#hide
# !pip install -q nbdev lambdasdk s3bz

In [1]:
#hide
USER=None
PW=None

# Products

> interact with product apis

In [2]:
#hide
from nbdev.showdoc import *
import logging

In [3]:
#export
from botocore.config import Config
from s3bz.s3bz import S3, Requests
from lambdasdk.lambdasdk import Lambda, InvocationType
from villaProductSdk.schema import Event, Response
from typing import Optional, List
import ujson as json
from nicHelper.wrappers import add_method
from nicHelper.exception import errorString
from awsSchema.apigateway import Event, Response
from dataclasses_json import dataclass_json
from dataclasses import dataclass
import bz2,  boto3, base64, logging, requests

In [4]:
#hide
logging.basicConfig(level=logging.INFO)

In [16]:
#export
INTCOLS = ['iprcode','cprcode', 'oprcode', 'pr_barcode', 'pr_barcode2', 'sellingPrice']



In [5]:
#export
class FunctionNames:
  '''determine function and resources name based on branchName'''
  def __init__(self, branchName:str = 'dev-manual'):
    self.branchName = branchName
  dumpToS3 = lambda self: f'product-dump-s3-{self.branchName}'
  updateProduct = lambda self: f'product-update-{self.branchName}'
  updateS3 = lambda self: f'product-update-s3-{self.branchName}'
  singleQuery = lambda self: f'product-get-{self.branchName}'
  allQuery = lambda self: f'product-get-all-{self.branchName}'
  inputBucket = lambda self: f'input-product-bucket-{self.branchName}'
  inventoryBucket = lambda self: f'product-bucket-{self.branchName}'
  listQuery = lambda self: f'product-get-list-{self.branchName}'
  

In [50]:
#export
class ProductSdk:
  '''
    the main class for interacting with product endpoint
    user/pw are optional
  '''
  def __init__(self, 
               branch:str = 'dev-manual', 
               user:Optional[str] = None, 
               pw:Optional[str] = None,
               region:str = 'ap-southeast-1'):
    self.branchName = branch
    self.functionNames = FunctionNames(branchName = branch)
    self.lambdaClient = Lambda(user =user, pw=pw, region = region)
    self.user = user; self.pw = pw; self.region = region
    
  @staticmethod
  def returnLambdaResponse(lambdaResponse:dict):
    try:
      return Response.fromDict(lambdaResponse).body
    except:
      logging.exception(f'error parsing body, perhaps there is no body in response\
      response is {lambdaResponse}')
      logging.error(lambdaResponse)
      
  @staticmethod
  def printFirst(inputDict:dict):
    return next(iter(inputDict.items()))
  
  def generalInvoke(self, functionName, payload):
    lambdaResponse = self.lambdaClient.invoke(
      functionName = functionName, input = payload
    )
    try:
      response:Response = Response.fromDict(lambdaResponse)
    except:
      print('unable to parse response')
      print(lambdaResponse)
      raise Exception(errorString())
    if response.statusCode == 200:
      return response.body
    else:
      print(f'error{response}')
      return response.body
    return self.returnLambdaResponse(lambdaResponse)
    
  def updateWithS3(self, data, 
                   inputKeyName = 'input-data-name', 
                   invocationType = InvocationType.event,
                   user= None, pw= None):
    # put users if not specified
    user = user or self.user; pw = pw or self.pw
    
    # extract function name and inputbucket name
    inputBucketName = self.functionNames.inputBucket()
    functionName = self.functionNames.updateS3()
    logging.info(f'bucket is {inputBucketName}')
    
    # save data to s3
    ## convert keys to int
    filteredProducts = [{k:int(v) if k in INTCOLS else str(v) for k,v in product.items()}for product in data]
    
    S3.save(key = inputKeyName, 
            objectToSave = filteredProducts , 
            bucket = inputBucketName,
            user=user, pw=pw)
    logging.info(f'data is saved to s3, invoking ingestion function')
    
    # call lambda function
    inputValue = Event(body = json.dumps({ 'key': inputKeyName })).to_dict()
    logging.info(f'input to lambda is {inputValue}')
    lambdaResponse = self.lambdaClient.invoke(
      functionName= functionName ,
      input=inputValue, 
      invocationType= invocationType )
    logging.info(f'lambdaResponse is {lambdaResponse}')
    if invocationType == 'Event': return "successfully sent event, please watch your slack"
    if lambdaResponse: return self.returnLambdaResponse(lambdaResponse)

  
  def allQuery(self):
    functionName = self.functionNames.allQuery()
    lambdaResponse = self.lambdaClient.invoke(
      functionName = functionName, input = {}
    )
    url = Response.fromDict(lambdaResponse).body['url']
    result = retrieveJsonZl(url)
    return result
  def syncS3(self):
    '''force s3 to sync with the newly input data'''
    functionName = self.functionNames.dumpToS3()
    lambdaResponse = self.lambdaClient.invoke(
      functionName = functionName, input = {}
    )
    return self.returnLambdaResponse(lambdaResponse)


In [51]:
#export
import zlib
def retrieveJsonZl(url):
  response:requests.Response = requests.get(url)
  if response.status_code != 200:
    print(response.json())
  else:
    content = response.content
    jsonDb = zlib.decompress(content).decode()
    newDb = pd.read_json(jsonDb,orient='split')
    return newDb

# Testing

In [57]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from random import randrange
from datetime import datetime
from pprint import pprint
import boto3
import pandas as pd


## generate dummy data for testing

In [58]:
#Dummy Data
sampleProducts = [{'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAPAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845     ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0235141', 'iprcode': '0235141', 'oprcode': '0235141', 'ordertype': 'Y', 'pr_abb': 'EEBOO-PZCT3-PUZZLE', 'pr_active': 'Y', 'pr_cgcode': '08', 'pr_code': '0235141', 'pr_dpcode': '19', 'pr_engname': 'EEBOO,ANIMAL COUNTING PUZZLE_3ED,PZCT3', 'pr_ggcode': '113', 'pr_market': 'eeboo,PUZZLE-PZCT3', 'pr_name': 'EEBOO-PZCT3-ตัวต่อนับเลข ANIMAL COUNTING_3ED', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM1979    ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0217153', 'iprcode': '0217153', 'oprcode': '0217153', 'ordertype': 'Y', 'pr_abb': 'COCOA LOCO MILK CHOC', 'pr_active': 'Y', 'pr_cgcode': '98', 'pr_code': '0217153', 'pr_dpcode': '28', 'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_ggcode': '003', 'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL', 'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_puqty': '24', 'pr_sa_method': '1', 'pr_sucode1': 'F1222     ', 'pr_suref3': 'S', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0182223', 'iprcode': '0182223', 'oprcode': '0182223', 'ordertype': 'Y', 'pr_abb': 'CIRIO PIZZASSIMO 400', 'pr_active': 'Y', 'pr_cgcode': '06', 'pr_code': '0182223', 'pr_dpcode': '06', 'pr_engname': 'CIRIO PIZZASSIMO 400G.', 'pr_ggcode': '004', 'pr_market': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_name': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '2589      ', 'pr_suref3': 'C', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0124461', 'iprcode': '0124461', 'oprcode': '0124461', 'ordertype': 'Y', 'pr_abb': 'NEW CHOICE LYCHEE', 'pr_active': 'Y', 'pr_cgcode': '02', 'pr_code': '0124461', 'pr_dpcode': '02', 'pr_engname': 'NEW CHOICE LYCHEE', 'pr_ggcode': '003', 'pr_market': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_name': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '695       ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]
# sampleProducts = [{k: int(v) if k in INTCOLS else v for k,v in product.items()}for product in sampleProducts]

## Create main class object

In [59]:
sdk = ProductSdk(branch = 'dev-manual')

## Upload s3 data

In [60]:
#hide

In [61]:
%%time
result = sdk.updateWithS3(
    sampleProducts,
    invocationType = InvocationType.requestResponse
  )
if 'error' in result.keys():
  print(result['error'])
else:
  print(result)

INFO:root:bucket is input-product-bucket-dev-manual
INFO:root:using accelerate endpoint
INFO:root:data was saved to s3
INFO:root:data is saved to s3, invoking ingestion function
INFO:root:input to lambda is {'body': '{"key":"input-data-name"}', 'headers': {}, 'statusCode': 200}
INFO:root:lambdaResponse is {'body': '{"success":5,"failure":0,"skipped":0,"failureMessage":[],"timetaken(ms)":269.98}', 'statusCode': 200, 'headers': {'Access-Control-Allow-Headers': '*', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': '*'}}


{'success': 5, 'failure': 0, 'skipped': 0, 'failureMessage': [], 'timetaken(ms)': 269.98}
CPU times: user 37.6 ms, sys: 3.83 ms, total: 41.4 ms
Wall time: 1.16 s


### test with lots of data

In [44]:
df = pd.read_csv('./sampleData/products.csv').fillna('none').astype('string')
for col in df.columns:
  if col in INTCOLS:
    df[col] = df[col].astype(int)
data = json.loads(df.to_json(orient='records'))
data[:1]

# data = list(pd.read_csv('./sampleData/products.csv').fillna('none').astype('string').T.to_dict().values())

/home/ec2-user/SageMaker/.persisted_conda/python38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,7,10,20,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


[{'cprcode': 9,
  'iprcode': 9,
  'oprcode': 9,
  'ordertype': 'Y',
  'pr_abb': 'J. WALKER RED 70 CL.',
  'pr_active': 'Y',
  'pr_cgcode': '1',
  'pr_code': '9',
  'pr_dpcode': '1.0',
  'pr_engname': 'JOHNNIE WALKER RED 70 CL.',
  'pr_ggcode': '1',
  'pr_market': 'J. WALKER RED 70 CL',
  'pr_name': 'JOHNNIE WALKER RED 70 CL',
  'pr_puqty': '12.0',
  'pr_sa_method': '1',
  'pr_sucode1': '1782',
  'pr_suref3': 'A',
  'prtype': 'I',
  'psqty': '1.0',
  'pstype': '1.0',
  'pr_country_th': 'none',
  'pr_country_en': 'United Kingdom',
  'pr_keyword_th': 'none',
  'pr_keyword_en': 'none',
  'pr_filter_th': 'Alcohol,Spirits  Liqueurs',
  'pr_filter_en': 'Alcohol,Spirits  Liqueurs',
  'online_category_l1_th': 'เบียร์, ไวน์และสุรา',
  'online_category_l1_en': 'Beer Wine & Spirits',
  'online_category_l2_th': 'เหล้า',
  'online_category_l2_en': 'Spirits & Liqueurs',
  'online_category_l3_th': 'none',
  'online_category_l3_en': 'none',
  'villa_category_l1_en': 'Dry Grocery',
  'villa_category_l2_

In [46]:
%%time
result = sdk.updateWithS3(
    data,
    invocationType = InvocationType.event
  )
result

INFO:root:bucket is input-product-bucket-dev-manual
INFO:root:using accelerate endpoint
INFO:root:data was saved to s3
INFO:root:data is saved to s3, invoking ingestion function
INFO:root:input to lambda is {'body': '{"key":"input-data-name"}', 'headers': {}, 'statusCode': 200}
INFO:root:lambdaResponse is True


CPU times: user 17.3 s, sys: 272 ms, total: 17.5 s
Wall time: 17.8 s


'successfully sent event, please watch your slack'

In [32]:
sdk.allQuery()

,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,...,pr_keyword_th,pr_name_en,pr_name_th,pr_online_name_en,pr_online_name_th,sort_weight,villa_category_l1_en,villa_category_l2_en,villa_category_l3_en,villa_category_l4_en
0,182223,182223,182223,Y,CIRIO PIZZASSIMO 400,Y,6,182223,6,CIRIO PIZZASSIMO 400G.,...,None,None,None,None,None,NaN,None,None,None,None
1,9,9,9,Y,J. WALKER RED 70 CL.,Y,1,9,1,JOHNNIE WALKER RED 70 CL.,...,none,JOHNNIE WALKER RED 70 CL.,JOHNNIE WALKER RED 70 CL,Johnnie Walker Red 700ml,JOHNNIE WALKER RED 700ml,0.0,Dry Grocery,Alcohol & Beverage,Whisky,Scoth Whisky
2,235141,235141,235141,Y,EEBOO-PZCT3-PUZZLE,Y,8,235141,19,"EEBOO,ANIMAL COUNTING PUZZLE_3ED,PZCT3",...,None,None,None,None,None,NaN,None,None,None,None
3,217153,217153,217153,Y,COCOA LOCO MILK CHOC,Y,98,217153,28,COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.,...,None,None,None,None,None,NaN,None,None,None,None
4,28,28,28,Y,J B RARE WHISKY 750,Y,1,28,1,J B RARE WHISKY 750 ML.,...,none,J B RARE WHISKY 750 ML.,J B RARE WHISKY 750 ML,J&B Rare Whisky 750ml,J&B Rare Whisky 750ml,0.0,Dry Grocery,Alcohol & Beverage,Whisky,Scoth Whisky
5,12,12,12,Y,J.WALER BLACK 70 CL,Y,1,12,1,JOHNNIE WALKER BLACK 70 CL.,...,none,JOHNNIE WALKER BLACK 70 CL.,JOHNNIE WALKER BLACK 70 CL.,Johnnie Walker Black 70 Cl.,JOHNNIE WALKER BLACK 70 CL.,0.0,Dry Grocery,Alcohol & Beverage,Whisky,none
6,33,33,33,Y,BALLANTINE S 700ml.,Y,1,33,1,BALLANTINE S FINEST 700 ML.,...,none,BALLANTINE S FINEST 700 ML.,BALLANTINE S FINEST 700 ML,Ballentine's Finest Whisky 700ml,Ballentine's Finest Whisky 700ml,0.0,Dry Grocery,Alcohol & Beverage,Whisky,none
7,26,26,26,Y,JOHN JAMESON 70 CL,Y,1,26,1,JOHN JAMESON 70 CL WHISKY,...,none,JOHN JAMESON 70 CL WHISKY,JOHN JAMESON 70 CL WHISKY,John Jameson Whisky 700ml,John Jameson Whisky 700ml,0.0,Dry Grocery,Alcohol & Beverage,Whisky,none
8,124461,124461,124461,Y,NEW CHOICE LYCHEE,Y,2,124461,2,NEW CHOICE LYCHEE,...,None,None,None,None,None,NaN,None,None,None,None
9,171670,171670,171670,Y,JIRAPAT YOUG KALE 2,Y,5,171670,19,JIRAPAT YOUNG KALE 200 G.,...,None,None,None,None,None,NaN,None,None,None,None


## Query Single Product

In [23]:
#export
@add_method(ProductSdk)
def querySingleProduct(self, iprcode = '0171670', user=None, pw=None):
  '''query a single product'''
  #extract function name
  functionName = self.functionNames.singleQuery()
  query = {'iprcode': iprcode}
  try:
    inputValue = Event.getInput(query)
    lambdaResponse = self.lambdaClient.invoke( functionName = functionName , input = inputValue )
    return self.returnLambdaResponse(lambdaResponse)
  except:
    print('calling lambda failed')


In [24]:
result = sdk.querySingleProduct('0171670')
result

{'cprcode': '0171670',
 'iprcode': '0171670',
 'oprcode': '0171670',
 'ordertype': 'Y',
 'pr_abb': 'JIRAPAT YOUNG KALE 2',
 'pr_active': 'Y',
 'pr_cgcode': '05',
 'pr_code': '0171670',
 'pr_dpcode': '19',
 'pr_engname': 'JIRAAT YOUNG KALE 200 G.',
 'pr_ggcode': '057',
 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.',
 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.',
 'pr_puqty': '1',
 'pr_sa_method': '1',
 'pr_sucode1': 'CM845',
 'pr_suref3': 'A',
 'prtype': 'I',
 'psqty': '1',
 'pstype': '1',
 'pr_country_th': 'none',
 'pr_country_en': 'none',
 'pr_keyword_th': 'none',
 'pr_keyword_en': 'none',
 'pr_filter_th': 'none',
 'pr_filter_en': 'none',
 'online_category_l1_th': 'none',
 'online_category_l1_en': 'none',
 'online_category_l2_th': 'none',
 'online_category_l2_en': 'none',
 'online_category_l3_th': 'none',
 'online_category_l3_en': 'none',
 'villa_category_l1_en': 'none',
 'villa_category_l2_en': 'none',
 'villa_category_l3_en': 'none',
 'villa_category_l4_en': 'none',
 'content_en': '0171670 JIRAPAT

In [25]:
## failure query product notn found
result = sdk.querySingleProduct('3203290')
result

{'error': 'product not found'}

## All Query

In [49]:
result = sdk.allQuery()
print(result.shape)
result.head()

(711, 50)


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,...,pr_keyword_th,pr_name_en,pr_name_th,pr_online_name_en,pr_online_name_th,sort_weight,villa_category_l1_en,villa_category_l2_en,villa_category_l3_en,villa_category_l4_en
0,182223,182223,182223,Y,CIRIO PIZZASSIMO 400,Y,6,182223,6,CIRIO PIZZASSIMO 400G.,...,None,None,None,None,None,NaN,None,None,None,None
1,9,9,9,Y,J. WALKER RED 70 CL.,Y,1,9,1,JOHNNIE WALKER RED 70 CL.,...,none,JOHNNIE WALKER RED 70 CL.,JOHNNIE WALKER RED 70 CL,Johnnie Walker Red 700ml,JOHNNIE WALKER RED 700ml,0.0,Dry Grocery,Alcohol & Beverage,Whisky,Scoth Whisky
2,235141,235141,235141,Y,EEBOO-PZCT3-PUZZLE,Y,8,235141,19,"EEBOO,ANIMAL COUNTING PUZZLE_3ED,PZCT3",...,None,None,None,None,None,NaN,None,None,None,None
3,217153,217153,217153,Y,COCOA LOCO MILK CHOC,Y,98,217153,28,COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.,...,None,None,None,None,None,NaN,None,None,None,None
4,28,28,28,Y,J B RARE WHISKY 750,Y,1,28,1,J B RARE WHISKY 750 ML.,...,none,J B RARE WHISKY 750 ML.,J B RARE WHISKY 750 ML,J&B Rare Whisky 750ml,J&B Rare Whisky 750ml,0.0,Dry Grocery,Alcohol & Beverage,Whisky,Scoth Whisky


## Query list of items

In [63]:
#export
@dataclass_json
@dataclass
class ProductsFromList:
  iprcodes: List[str]

In [64]:
#export
@add_method(ProductSdk)
def queryList(self,iprcodes:List[str])->List[dict]:
#   return self.generalInvoke(functionName=self.functionNames.listQuery() ,payload=Event.getInput({'iprcodes': iprcodes}))
  response = self.lambdaClient.invoke(functionName=self.functionNames.listQuery(),input=Event.getInput({'iprcodes':iprcodes}))
  print(response)
  

In [65]:
cprcodes = ['0217153','203915','0000009']
sdk.queryList(cprcodes)

{'errorMessage': "'cprcode'", 'errorType': 'KeyError', 'stackTrace': ['  File "/opt/python/villaProductDatabase/database.py", line 104, in lambdaProductsFromList\n    result:pd.DataFrame = ProductDatabase.productsFromList(productsFromList.iprcodes)\n', '  File "/opt/python/nicHelper/wrappers.py", line 37, in wrapper\n    return func( cls, *args, **kwargs)\n', '  File "/opt/python/villaProductDatabase/query.py", line 66, in productsFromList\n    products = db[db[\'cprcode\'].isin(cprcodes)]\n', '  File "/opt/python/pandas/core/frame.py", line 2906, in __getitem__\n    indexer = self.columns.get_loc(key)\n', '  File "/opt/python/pandas/core/indexes/base.py", line 2900, in get_loc\n    raise KeyError(key) from err\n']}


In [69]:
json.dumps(Event.getInput({'iprcodes':iprcodes}))

'{"body":"{\\"iprcodes\\":[\\"0217153\\",\\"203915\\",\\"0000009\\"]}","headers":{},"statusCode":200}'

## Trigger s3 sync

In [47]:
%%time
response = sdk.syncS3()
response

CPU times: user 15 ms, sys: 79 µs, total: 15 ms
Wall time: 7.6 s


{'result': {'cprcode': 182223,
  'iprcode': 182223,
  'oprcode': 182223,
  'ordertype': 'Y',
  'pr_abb': 'CIRIO PIZZASSIMO 400',
  'pr_active': 'Y',
  'pr_cgcode': '06',
  'pr_code': '0182223',
  'pr_dpcode': '06',
  'pr_engname': 'CIRIO PIZZASSIMO 400G.',
  'pr_ggcode': '004',
  'pr_market': 'CIRIO ซอสทำพิซซ่า 400 G.',
  'pr_name': 'CIRIO ซอสทำพิซซ่า 400 G.',
  'pr_puqty': '12',
  'pr_sa_method': '1',
  'pr_sucode1': '2589',
  'pr_suref3': 'C',
  'prtype': 'I',
  'psqty': '1',
  'pstype': '1',
  'content_en': None,
  'content_th': None,
  'hema_brand_en': None,
  'hema_brand_th': None,
  'hema_name_en': None,
  'hema_name_th': None,
  'hema_sizedesc': None,
  'online_category_l1_en': None,
  'online_category_l1_th': None,
  'online_category_l2_en': None,
  'online_category_l2_th': None,
  'online_category_l3_en': None,
  'online_category_l3_th': None,
  'pr_brand_en': None,
  'pr_brand_th': None,
  'pr_country_en': None,
  'pr_country_th': None,
  'pr_filter_en': None,
  'pr_filter_th

In [ ]:
#hide
from nbdev.export import *
notebook2script()

In [ ]:
%time
import requests
from io import BytesIO
from PIL import Image

##large Image
url = 'http://d19oj5aeuefgv.cloudfront.net'
sku = '0189194'
Image.open(BytesIO(requests.get(f'{url}/{sku}').content),formats=['PNG'])

In [ ]:
%time
url = 'https://d1vl5j0v241n75.cloudfront.net'
sku = '0189194'
Image.open(BytesIO(requests.get(f'{url}/{sku}').content),formats=['PNG'])